# API Scavenger Game

## Challenge 1: Fork Languages

You will find out how many programming languages are used among all the forks created from the main lab repo of your bootcamp.

In [5]:
# import libraries here
import requests
import pandas as pd
import re
import json

username = 'IrynaHorova'
%run mytoken.ipynb
#token = 'your token'

# response = requests.get('https://api.github.com/user/repos?page=1&per_page=100', auth=(username,token))
# data = pd.DataFrame(response.json())
# data

Assuming the main lab repo is ironhack-datalabs/madrid-oct-2018, you will:

#### 1. Obtain the full list of forks created from the main lab repo via Github API.

To list forks, we can use the GET method. As explained in the GitHub API documentation, we need to make the request to: GET /repos/:owner/:repo/forks.

In [6]:
# your code /repos/:owner/:repo/forks
response = requests.get("https://api.github.com/repos/jmolendijk/dataV2-labs/forks", auth = (username,token))
#https://api.github.com/repos/jmolendijk/dataV2-labs/forks?sort=newest
data = pd.DataFrame(response.json())
print('List of forks:')
print(data['full_name'])

forks = data['full_name']

List of forks:
0      MichelleMartens/dataV2-labs
1          tsharma2020/dataV2-labs
2     SvetlanaGruzdeva/dataV2-labs
3           iswetha522/dataV2-labs
4        Sebastieniron/dataV2-labs
5          IrynaHorova/dataV2-labs
6            Laurannvv/dataV2-labs
7            AjiacoVic/dataV2-labs
8        leonvanderlee/dataV2-labs
9              Vika182/dataV2-labs
10              B3rtje/dataV2-labs
11             NellyCR/dataV2-labs
12        sarah-busson/dataV2-labs
Name: full_name, dtype: object


#### 2. Loop the JSON response to find out the language attribute of each fork. Use an array to store the language attributes of each fork.
Hint: Each language should appear only once in your array.
Print the language array. It should be something like: ["Python", "Jupyter Notebook", "HTML"]

In [7]:
# your code here
print("Languages:")
list(data['language'].unique())

Languages:


['Jupyter Notebook']

## Challenge 2: Count Commits
Count how many commits were made in the month of october of 2018.
#### 1. Obtain all the commits made in October 2018 via API, which is a JSON array that contains multiple commit objects.

In [8]:
# your code here
#I am going to look at July 2020 commits - in order to be able to count the commits
response = requests.get(f"https://api.github.com/repos/jmolendijk/dataV2-labs/commits?since=2020-07-01T00:00:00Z&until=2020-07-31T00:00:00Z", auth = (username,token))
#response.json()

#### 2. Count how many commit objects are contained in the array.

In [9]:
commits = pd.DataFrame(response.json())
print(f"There {len(commits)} commit objects")

There 2 commit objects


## Challenge 3: Hidden Cold Joke

Using Python, call Github API to find out the cold joke contained in the 24 secret files in the following repo:

https://github.com/ironhack-datalabs/scavenger

The filenames of the secret files contain .scavengerhunt and they are scattered in different directories of this repo. The secret files are named from .0001.scavengerhunt to .0024.scavengerhunt. They are scattered randomly throughout this repo. You need to search for these files by calling the Github API, not searching the local files on your computer.

#### 1. Find the secret files.

In [21]:
# your code here
response = requests.get("https://api.github.com/repos/ironhack-datalabs/scavenger/contents", auth = (username,token))
cont = pd.DataFrame(response.json())
path  = list(cont['path'])

In [22]:
#path_df = df.DataFrame(columns = ['file_name', 'file_path'])
path_dict ={}
for pth in path[1:]:
    response = requests.get(f"https://api.github.com/repos/ironhack-datalabs/scavenger/contents/{pth}", auth = (username,token))
    response = pd.DataFrame(response.json())
    for i in range(len(response)) : 
        path_dict[response.loc[i, "name"]] = response.loc[i, "path"]

In [23]:
path_df = pd.DataFrame.from_dict(path_dict, orient='index',
                       columns=['path']).reset_index()

#### 2.  Sort the filenames ascendingly.

In [24]:
path_df = path_df.rename(columns = {'index':'file_name'})
path_df = path_df[path_df['file_name'].str.contains('.scavengerhunt')]
path_df = path_df.sort_values(by = ['file_name'])
path_df

,file_name,path
80,.0001.scavengerhunt,98750/.0001.scavengerhunt
62,.0002.scavengerhunt,88596/.0002.scavengerhunt
43,.0003.scavengerhunt,60224/.0003.scavengerhunt
52,.0004.scavengerhunt,68848/.0004.scavengerhunt
17,.0005.scavengerhunt,44639/.0005.scavengerhunt
0,.0006.scavengerhunt,15024/.0006.scavengerhunt
8,.0007.scavengerhunt,17020/.0007.scavengerhunt
4,.0008.scavengerhunt,15534/.0008.scavengerhunt
76,.0009.scavengerhunt,97881/.0009.scavengerhunt
27,.0010.scavengerhunt,47830/.0010.scavengerhunt


#### 3. Read the content of each secret files into an array of strings.
Since the response is encoded, you will need to send the following information in the header of your request:
````python
headers = {'Accept': 'application/vnd.github.v3.raw'}
````

In [57]:
# your code here

joke = []

for pth in path_df['path']:
    response = requests.get(f"https://api.github.com/repos/ironhack-datalabs/scavenger/contents/{pth}", headers = {'Accept': 'application/vnd.github.v3.raw'}, auth = (username,token))
    word = response.text #response.content previously; .text outputs string rather than bite class
    joke.append(word)

In [58]:
joke

['In\n',
 'data\n',
 'science,\n',
 '80\n',
 'percent\n',
 'of\n',
 'time\n',
 'spent\n',
 'is\n',
 'preparing\n',
 'data,\n',
 '20\n',
 'percent\n',
 'of\n',
 'time\n',
 'is\n',
 'spent\n',
 'complaining\n',
 'about\n',
 'the\n',
 'need\n',
 'to\n',
 'prepare\n',
 'data.\n']

#### 4. Concatenate the strings in the array separating each two with a whitespace.

In [15]:
# your code here
joke_parsed  = " ".join(joke)
joke_parsed = joke_parsed.replace("\n",'')

#### 5. Print out the joke.

In [16]:
# your code here
print(joke_parsed)

In data science, 80 percent of time spent is preparing data, 20 percent of time is spent complaining about the need to prepare data.
